Load some packages 

In [1]:
%run preample.ipy

Import specific functions 

In [2]:
from Utilities import *
from BiCM import *

# Numerical Solving BiCM
    - Get degree sequences
    - run numerical simulation to solve system
    - calculate the link probabilities
    

In [3]:
def get_degree_sequences(M_hat):
    
    degree_sequence_item = np.sum(M_hat,1)
    degree_sequence_product = np.sum(M_hat,0)
    degree_sequence = list(np.concatenate([degree_sequence_item, degree_sequence_product]))
    n_rows = len(degree_sequence_item)
    n_cols = len(degree_sequence_product)
    return degree_sequence, n_rows, n_cols

In [4]:
def get_link_probabilities(M, solver = 'root'):
    [degree_sequence, n_rows, n_cols] = get_degree_sequences(M)
    bicm = bicm_leastSquares(degree_sequence, n_rows, n_cols)
    if solver == 'root':
        bicm.solve_root()
    if solver == 'least_squares':
        bicm.solve_least_squares()
    if solver == 'trust_region':
        bicm.solve_trust_regions()
    bicm.generate_p_adjacency()
    return bicm.p_adjacency, abs(np.sum(bicm.final_result))

# WTW

Link removal

In [5]:
def remove_links(A, frac = 0.1):
    nonzero = zip(*np.nonzero(A))
    len_nonzeros = len(list(nonzero))
    nonzero = list(zip(*np.nonzero(A)))
    N_links_remove = int(len_nonzeros*frac)
    idx_to_remove = np.random.choice(len_nonzeros, N_links_remove, replace=False)

    A_hat = A.copy()
    for idx in idx_to_remove:
        A_hat[nonzero[idx]] = 0
    removed_links = [nonzero[i] for i in idx_to_remove]
    return A_hat, removed_links
 

In [6]:
adj = np.genfromtxt('../data/raw/Mcp_2000.dat')
adj = adj.astype(int)
np.savetxt('../data/intermediate/wtw/adjacency/wtw_2000.csv', adj)

file_dir = '../data/intermediate/wtw/adjacency_link_removed/'
for i in np.arange(15):
    adj_removed = remove_links(adj)[0]
    print(np.sum(adj))
    print(np.sum(adj_removed))
    np.savetxt(file_dir + 'wtw_2000_' + str(i) + '.csv', adj_removed)


21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132
21257
19132


Solving

In [ ]:

import os
file_dir = '../data/intermediate/wtw/adjacency_link_removed/'

for file in tqdm(os.listdir(file_dir)):
    print(file)
    adj = np.loadtxt(file_dir + file).astype(int)
    
    original_path = '../data/intermediate/wtw/adjacency/wtw_2000.csv'
    original_matrix = np.loadtxt(original_path)
    num_removed_links = np.sum(original_matrix) - np.sum(adj)
    
    p_adj, cost = get_link_probabilities(adj)
    print('solved', cost)
    if cost > 1:
        p_adj, cost = get_link_probabilities(adj, 'least_squares')
        print('solvedls', cost)
    if cost > 1:
        print('unsolved', file)
        continue
    np.savetxt('../data/intermediate/wtw/probabilities_adjacency/' + file, p_adj)
    assert cost < 1, 'numerical convergence problems'
    df = sort_probabilities(p_adj, adj)
    R = fill_miss_links(df, adj, num_removed_links, 0)
    np.savetxt('../data/intermediate/wtw/adjacency_reconstructed/' + file, R)
    np.savetxt('../data/intermediate/wtw/probabilities_adjacency/' + file, p_adj)
    


  0%|          | 0/15 [00:00<?, ?it/s]

wtw_2000_5.csv
solved 2.250977182427505e-12


  7%|▋         | 1/15 [00:03<00:53,  3.85s/it]

wtw_2000_9.csv
solved 5.169198402654729e-13


 13%|█▎        | 2/15 [00:05<00:40,  3.10s/it]

...

wtw_2000_14.csv
solved 149.31293065637539


 87%|████████▋ | 13/15 [05:53<02:08, 64.33s/it]

`xtol` termination condition is satisfied.
Function evaluations 1637, initial cost 1.0788e+06, final cost 4.1644e+03, first-order optimality 1.29e+05.
solvedls 44.19761213612094
unsolved wtw_2000_14.csv
wtw_2000_1.csv
solved 39.37295128816676


 93%|█████████▎| 14/15 [06:14<00:51, 51.54s/it]

`xtol` termination condition is satisfied.
Function evaluations 1617, initial cost 1.0640e+06, final cost 4.1669e+03, first-order optimality 1.63e+05.
solvedls 247.63792638697913
unsolved wtw_2000_1.csv
wtw_2000_11.csv
